In [ ]:
import os
import json
import torch
import random
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, load_dataset
from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer,
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
import bitsandbytes as bnb
from huggingface_hub import login
import wandb
from transformers import BitsAndBytesConfig

# Load API tokens
CONFIG = json.load(open('/home/jupyter/datasphere/project/tokens.json'))
login(token=CONFIG["HF_TOK"])
wandb.login(key=CONFIG['WANDB_API_KEY'])

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

def set_random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_random_seed()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-15 15:52:33.699373: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Token is valid (permission: write).
Your token has been saved to /tmp/xdg_cache/huggingface/token
Login successful


wandb: Currently logged in as: rostik58533 (mika5883). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [11]:
# Load dataset
# dataset = load_dataset("csv", data_files=["/home/jupyter/datasphere/project/rugec/data/art_gec_full.tsv"], sep="\t")
# dataset = dataset["train"].rename_columns({'correct': 'correct_sent', 'corrupt': 'corrupt_sent'})
# dataset = dataset.filter(lambda example: example['sentence1'].startswith('Ar'))
# Load additional datasets
def load_tsv(path):
    return pd.read_csv(path, sep="\t", index_col=None)

# dev = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.dev.tsv')
# train = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.train.tsv')
# test = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv')
# clang8 = load_tsv('/home/jupyter/datasphere/project/rugec/data/clang8_source_target_ru.spacy_tokenized.tsv')
# relco = load_tsv('/home/jupyter/datasphere/project/rugec/data/relco_filtered.tsv')
train = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.train.tsv')
test = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.test.tsv')
dev = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.dev.tsv')
train.corrupt_sent = train.corrupt_sent.map(lambda x: str(x))
test.corrupt_sent = test.corrupt_sent.map(lambda x: str(x))
dev.corrupt_sent = dev.corrupt_sent.map(lambda x: str(x))
train.correct_sent = train.correct_sent.map(lambda x: str(x))
test.correct_sent = test.correct_sent.map(lambda x: str(x))
dev.correct_sent = dev.correct_sent.map(lambda x: str(x))
# Combine datasets
# train_all = pd.concat([train.sample(frac=1, replace=True), clang8, relco], ignore_index=True)
# augmented_data = dataset.select(range(500000))

# Convert to Hugging Face Dataset format
fine_tune = {
    'train': Dataset.from_pandas(train),
    'test': Dataset.from_pandas(test),
    'dev': Dataset.from_pandas(dev)
}
# fine_tune['train'] = datasets.concatenate_datasets([fine_tune['train'], augmented_data])
fine_tune

{'train': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 4591
 }),
 'test': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 1314
 }),
 'dev': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 775
 })}

In [18]:
fine_tune['train'] = fine_tune['train'].shuffle()

In [12]:
for i in fine_tune['train'].take(2):
    print(i)

{'corrupt_sent': 'И никого не прокляну !', 'correct_sent': 'И никого не прокляну !'}
{'corrupt_sent': '( По поэме М. Ю. Лермонтова " Мцыри " . )', 'correct_sent': '( По поэме М. Ю. Лермонтова " Мцыри " . )'}


In [ ]:
# Load model and tokenizer
# model_name = "RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4"
model_name = 'Qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
).to(device)
model.config.use_cache = False

# LoRA configuration
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = {name.split(".")[-1] for name, module in model.named_modules() if isinstance(module, cls)}
    lora_module_names.discard("lm_head")
    return list(lora_module_names)

target_modules = find_all_linear_names(model)
peft_config = LoraConfig(
    lora_alpha=64,
    target_modules=target_modules,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM"
)

Loading checkpoint shards: 100%|██████████| 4/4 [04:13<00:00, 63.38s/it]


In [42]:
from transformers import BitsAndBytesConfig
from peft import PeftModel, PeftConfig
# Your adapter repo or local dir
# peft_model_id = "mika5883/ru_qwen_gec" # or your output_dir path
# peft_model_id = 'mika5883/ru_qwen_gec_Ag_art'
peft_model_id = 'mika5883/ru_qwen7b_gec_Ag'

# Load adapter config
peft_config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

# Merge LoRA adapter with base model
model = PeftModel.from_pretrained(base_model, peft_model_id, is_trainable=True)
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.35s/it]


In [40]:
# model = PeftModel.from_pretrained(base_model, peft_model_id)
model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [13]:
def format_dataset(examples):
    # Prepare the text for tokenization
    text = [
        f"<|im_start|>system\nТы полезный ассистент для исправления грамматических ошибок.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: {corrupt}\nИсправленное предложение:<|im_end|>\n<|im_start|>assistant\n{correct}<|im_end|>"
        for corrupt, correct in zip(examples['corrupt_sent'], examples['correct_sent'])
    ]
    
    # Tokenize the text and prepare the labels
    inputs = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples['correct_sent'], truncation=True, padding="max_length", max_length=512)

    # Add the labels to the inputs
    inputs['labels'] = labels['input_ids']
    
    return inputs

# Apply the formatting to all splits
fine_tune = {key: value.map(format_dataset, batched=True) for key, value in fine_tune.items()}


Map: 100%|██████████| 775/775 [00:00<00:00, 3502.35 examples/s]


In [14]:
for i in fine_tune['train'].take(1):
    print(i)

{'corrupt_sent': 'И никого не прокляну !', 'correct_sent': 'И никого не прокляну !', 'input_ids': [151644, 8948, 198, 33995, 4552, 28519, 31885, 33513, 20396, 128698, 34011, 18673, 19849, 52587, 26988, 61676, 24725, 37708, 125727, 130084, 88663, 14746, 13, 151645, 198, 151644, 872, 198, 30174, 32693, 26988, 4824, 88663, 16748, 5805, 92029, 71019, 10090, 56825, 81841, 83098, 25, 42796, 126073, 22107, 18658, 12281, 14746, 13695, 125467, 58062, 30174, 32693, 26988, 60398, 47050, 56825, 131923, 25, 151645, 198, 151644, 77091, 198, 30174, 126073, 22107, 18658, 12281, 14746, 13695, 125467, 753, 151645, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 1516

In [18]:
len(fine_tune['train']) // 8 // 2 + 1

287

In [27]:
from transformers import EarlyStoppingCallback

In [ ]:
args = SFTConfig(
    output_dir="ru_qwen7b_gec_Ga",
    eval_strategy="steps",
    eval_steps=9,
    logging_steps=9,
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduce batch size to prevent memory issues
    per_device_eval_batch_size=8,
    resume_from_checkpoint="last-checkpoint",
    weight_decay=0.05,
    gradient_accumulation_steps=2,
    # warmup_steps=200,
    save_total_limit=1,
    # num_train_epochs=1,
    max_steps=288,
    bf16=True, #I was using A100
    push_to_hub=True,
    hub_strategy="checkpoint",
    metric_for_best_model='loss',
    report_to = 'all',
    save_strategy='best',
    load_best_model_at_end=True,
    lr_scheduler_type="cosine", 
    warmup_ratio=0.05,
    dataset_text_field='text',
    max_seq_length=512,
    packing=False,
)


In [45]:
import logging
logging.getLogger("wandb").setLevel(logging.ERROR)

In [47]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=fine_tune['train'],
    eval_dataset=fine_tune['dev'],
    peft_config=peft_config,
    processing_class=tokenizer,
    args=args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Training
trainer.train()


  3%|▎         | 9/288 [00:24<12:16,  2.64s/it]

{'loss': 0.7221, 'grad_norm': 1.149951457977295, 'learning_rate': 1.2e-05, 'epoch': 0.03}



 99%|█████████▉| 96/97 [00:56<00:00,  1.67it/s]
                                               
100%|██████████| 97/97 [00:57<00:00,  1.71it/s]
                                               

{'eval_loss': 0.7347809672355652, 'eval_runtime': 58.0327, 'eval_samples_per_second': 13.355, 'eval_steps_per_second': 1.671, 'epoch': 0.03}


  6%|▋         | 18/288 [01:58<17:22,  3.86s/it]  

...


 99%|█████████▉| 96/97 [00:56<00:00,  1.67it/s]
                                                 A
100%|██████████| 97/97 [00:57<00:00,  1.71it/s]
                                               

{'eval_loss': 0.6373622417449951, 'eval_runtime': 58.1063, 'eval_samples_per_second': 13.338, 'eval_steps_per_second': 1.669, 'epoch': 1.0}


100%|██████████| 288/288 [49:45<00:00, 10.37s/it]

{'train_runtime': 2985.6714, 'train_samples_per_second': 1.543, 'train_steps_per_second': 0.096, 'train_loss': 0.6348927401834064, 'epoch': 1.0}


TrainOutput(global_step=288, training_loss=0.6348927401834064, metrics={'train_runtime': 2985.6714, 'train_samples_per_second': 1.543, 'train_steps_per_second': 0.096, 'total_flos': 1.0063972687989965e+17, 'train_loss': 0.6348927401834064, 'epoch': 1.0034843205574913})

* Needed to ensure the best checkpoint was used when pushing

In [ ]:
# best_model_checkpoint = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen_gec_Ag_art/checkpoint-14600'  # e.g., "ru_qwen_gec_Ag/checkpoint-1000"
# best_model_checkpoint = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen7b_gec_Ag/checkpoint-3400'
best_model_checkpoint = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen7b_gec_Ga/checkpoint-261'
model = AutoModelForCausalLM.from_pretrained(
    best_model_checkpoint,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
).to(device)
model.config.use_cache = False
# tokenizer = AutoTokenizer.from_pretrained(best_model_checkpoint)






Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:  25%|██▌       | 1/4 [00:02<00:06,  2.11s/it]

Loading checkpoint shards:  50%|█████     | 2/4 [00:03<00:03,  1.62s/it]

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:04<00:01,  1.46s/it]

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.46s/it]


In [48]:
from peft import PeftModel, PeftConfig
# Your adapter repo or local dir
# peft_model_id = "/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen_gec_Ag/checkpoint-1600"  # or your output_dir path
# peft_model_id = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen7b_gec_Ag/checkpoint-3400'
peft_model_id = '/home/jupyter/datasphere/project/rugec/notebooks/qwen_gec/ru_qwen7b_gec_Ga/checkpoint-261'

# Load adapter config
peft_config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

# Merge LoRA adapter with base model
model = PeftModel.from_pretrained(base_model, peft_model_id)


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.51s/it]


In [49]:
trainer = SFTTrainer(
    model=model,
    train_dataset=fine_tune['train'],
    eval_dataset=fine_tune['dev'],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

/tmp/ipykernel_5865/3226559442.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


In [50]:
trainer.evaluate(fine_tune['test'])

100%|██████████| 165/165 [01:38<00:00,  1.68it/s]


{'eval_loss': 0.6015573143959045,
 'eval_model_preparation_time': 0.0063,
 'eval_runtime': 98.7911,
 'eval_samples_per_second': 13.301,
 'eval_steps_per_second': 1.67}

In [52]:
trainer.push_to_hub("qwen7b_Ga_v1")


events.out.tfevents.1744734694.g21-e3f0fcaa-10c4-3fbf-8de5-9e5c8efdc897.5865.2:   0%|          | 0.00/21.6k [00:00<?, ?B/s]
events.out.tfevents.1744734694.g21-e3f0fcaa-10c4-3fbf-8de5-9e5c8efdc897.5865.2: 100%|██████████| 21.6k/21.6k [00:00<00:00, 38.2kB/s]


CommitInfo(commit_url='https://huggingface.co/mika5883/ru_qwen7b_gec_Ga/commit/99fb11753b462630cf74e33afec877d3b1483ed6', commit_message='qwen7b_Ga_v1', commit_description='', oid='99fb11753b462630cf74e33afec877d3b1483ed6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mika5883/ru_qwen7b_gec_Ga', endpoint='https://huggingface.co', repo_type='model', repo_id='mika5883/ru_qwen7b_gec_Ga'), pr_revision=None, pr_num=None)